<a href="https://colab.research.google.com/github/gendeb/cloud-computing-specialization/blob/master/Fraud_detection_with_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
ealtman2019_credit_card_transactions_path = kagglehub.dataset_download('ealtman2019/credit-card-transactions')

print('Data source import complete.')


## Fraud Detection Using GNNs

## What is fraud detection ?

Fraud detection is the process of identifying and preventing unauthorized activity in organizations. It has become a major challenge for companies in industries such as banking, finance, retail, and e-commerce. Fraud can negatively impact an organization's financial performance and reputation, making it crucial for companies to prevent and predict suspicious activity.



## Complex and evolving fraud patterns

Fraudsters continually revise their strategies and devise ever-more-sophisticated methods to cheat the system, regularly involving intricate networks of transactions to prevent recognition. Traditional rule-based systems and matrix-based ML such as SVMs and XGBoost often just ponder the immediate edges of a deal (who transferred money to whom), usually omitting to notice fraudulent patterns with more complicated context. Rule-based systems furthermore need to be manually adjusted as fraud patterns shift and new scams arise.


## Label Quality

The datasets for fraud detection are commonly unbalanced and incompletely labeled. In reality, only a tiny fraction of people plan to perpetrate fraud. Usually, domain specialists will label transactions as fraudulent or not, but they can't make certain that all fraud has been identified in the dataset.

The large disparity between the number of labels and the amount of data available renders it hard to generate supervised models. Models trained on the accessible classifications could have a greater number of false negatives, while the imbalanced dataset could lead to an increase in false positives. Consequently, training Graph Neural Networks (GNNs) with different objectives and making use of their latent representations afterwards can be advantageous.


## Model explainability

Predicting if a transaction is fraudulent or not is inadequate for meeting the transparency standards of the banking sector. It is also essential to comprehend why certain deals are marked as deceitful. This explicability is critical for comprehending how fraud occurs, how to execute procedures to reduce fraud, and to make sure the process isn’t prejudiced. As a result, fraud detection models must be comprehensible and interpretable, which restricts the choices of models that investigators can apply.

## Graph approaches for fraud detection

Transactions can be represented as a graph, with users as nodes and interactions between them as edges. Traditional feature-based algorithms such as XGBoost and DLRM analyze individual nodes or edges, while graph-based approaches take into account the local context and structure of the graph, including neighboring nodes and edges.

In the traditional graph domain, there are various methods for making predictions based on graph structure, such as statistical approaches that aggregate features from adjacent nodes or edges. Algorithms like the [Louvain method](https://en.wikipedia.org/wiki/Louvain_method) and [InfoMap](https://towardsdatascience.com/infomap-algorithm-9b68b7e8b86) can identify communities and clusters of users on the graph. However, these methods lack the expressivity to fully consider the graph in its original format.

Graph neural networks (GNNs) offer a solution by representing local structural and feature context natively within the model. Through aggregation and message passing, information from both edges and nodes is propagated to neighboring nodes. Multiple layers of graph convolution result in a node's state containing information from nodes multiple layers away, effectively allowing the GNN to have a "receptive field" of nodes or edges multiple jumps away from the node or edge in question.

For fraud detection, GNNs can account for complex or longer chains of transactions that fraudsters use for obfuscation. Additionally, changing patterns can be addressed by retraining the model iteratively.

## ML for graph

These are the general steps of ML on Graph, depending on the task, the methods used in each step may change and not all the steps may be needed.

1. Graph Representation: The first step in applying machine learning techniques to graphs is to represent the graph in a format that can be input into a machine learning model. This can be done using various graph representations such as adjacency matrices, adjacency lists, and edge lists.

2. Feature Extraction: Once the graph is represented, the next step is to extract features from the graph that can be used as input to a machine learning model. These features can include things like node degree, centrality measures, and graph traversal patterns.

3. Dimensionality Reduction: As the number of nodes and edges in a graph can be large, it is often necessary to reduce the dimensionality of the graph representation before applying machine learning techniques. This can be done using techniques like principal component analysis (PCA) and t-SNE.

4. Model Selection: After the graph is represented and features have been extracted, the next step is to select a machine learning model that can be applied to the graph data. Common models used for graph data include decision trees, random forests, and neural networks.

5. Training and Evaluation: Once the model is selected, it is trained on the graph data using a labeled training dataset. The model is then evaluated on a separate test dataset to assess its performance.

6. Predictions: After the model is trained, it can be used to make predictions on new graph data. For example, it can be used to predict the likelihood of a node in a graph being in a certain class or to predict the likelihood of a link between two nodes.

## Dataset
The IBM credit card transaction dataset is a publicly available dataset that contains information about credit card transactions. It is often used for research and testing of fraud detection models. The dataset includes a variety of features such as the amount of the transaction, the type of card used, and the location of the transaction. It also includes a label indicating whether or not the transaction was fraudulent. The dataset is designed to be representative of real-world transactions and therefore contains a certain level of class imbalance, with a higher number of non-fraudulent transactions than fraudulent transactions. The dataset is provided by IBM and the data is simulated, but it is not specified the exact process of data simulation. It is important to note that the data is not real and it is not linked to any real customer or financial institution.

The data set contains:
* 24 million unique transactions
* 6,000 unique merchants
* 100,000 unique cards
* 30,000 fraudulent samples (0.1% of total transactions)

**Where to find the data**
* If you are willing to use the data locally you can download it from this [link](https://ibm.ent.box.com/v/tabformer-data)
* In our case we will use Kaggle to create our GNN model so we are using the dataset on kaggle, here's the [link](https://www.kaggle.com/datasets/ealtman2019/credit-card-transactions)

## Necessary imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F

## Preprosessing

Preprocessing tabular data before creating a graph is important because it ensures that the data is in a format that can be easily converted into a graph structure. Tabular data, such as a CSV file, typically contains rows and columns of information, with each row representing a single record and each column representing a specific feature or attribute. Before creating a graph, it is essential to clean and transform the data so that it is consistent and in a format that can be easily mapped to a graph structure. This may include tasks such as filling in missing values, converting data types, and removing duplicates.

Additionally, preprocessing tabular data can also help to improve the performance and accuracy of graph-based algorithms by reducing noise and irrelevant information in the data. This can be achieved by removing irrelevant features, normalizing numerical data, and encoding categorical data. Preprocessing can also help to balance the class distribution in case of imbalanced data, which is a common problem in fraud detection datasets.

**Note:**

For hardware limitations we will only take 100.000 samples of our data.

In [ ]:
# Load the tabular dataset
df = pd.read_csv("/kaggle/input/credit-card-transactions/credit_card_transactions-ibm_v2.csv").sample(n=100000, random_state=42)

In [ ]:
df.info()

In [ ]:
# We need to make sure that the samples we extracted have some rows where fraud is True
df [df['Is Fraud?'] == 'Yes'].shape

In [ ]:
percent_missing=(df.isnull().sum()*100/df.shape[0]).sort_values(ascending=True)
plt.title("Missing Value Analysis")
plt.xlabel("Features")
plt.ylabel("% of missing values")
plt.bar(percent_missing.sort_values(ascending=False).index,percent_missing.sort_values(ascending=False),color=(0.1, 0.1, 0.1, 0.1),edgecolor='blue')
plt.xticks(rotation=90)

The card_id is defined as one card by one user. A specific user can have multiple cards, which would correspond to multiple different card_ids for this graph.
For this reason we will create a new column which is the concatenation of the column User and the Column Card

In [ ]:
df["card_id"] = df["User"].astype(str) + "_" + df["Card"].astype(str)

In [ ]:
df.Amount.head(5)

In [ ]:
# We need to strip the ‘$’ from the Amount to cast as a float
df["Amount"]=df["Amount"].str.replace("$","").astype(float)

In [ ]:
# time can't be casted to int so so opted to extract the hour and minute
df["Hour"] = df["Time"].str [0:2]
df["Minute"] = df["Time"].str [3:5]

In [ ]:
df.Hour

In [ ]:
df.Minute

In [ ]:
df = df.drop(["Time","User","Card"],axis=1)
df.info()

In [ ]:
df["Errors?"].unique()

In [ ]:
df["Errors?"]= df["Errors?"].fillna("No error")

The two columns Zip and Merchant state contains missing values which can affect our graph. Moreover these information can be extracted from the column Merchant City so we will drop them.

In [ ]:
df = df.drop(columns=["Merchant State","Zip"],axis=1)

In [ ]:
# change the is fraud column to binary
df["Is Fraud?"] = df["Is Fraud?"].apply(lambda x: 1 if x == 'Yes' else 0)

In [ ]:
df["Merchant City"]=LabelEncoder().fit_transform(df["Merchant City"])
df["Use Chip"]=LabelEncoder().fit_transform(df["Use Chip"])
df["Errors?"]=LabelEncoder().fit_transform(df["Errors?"])

In [ ]:
df["Errors?"].unique()

In [ ]:
df.info()

## GNN for fraud detection:
Creating a multigraph for fraud detection using transaction data and applying a Graph Neural Network (GNN) on the edge list can be done in the following steps:

1. Prepare the transaction data: Collect and organize the transaction data into a format that can be used to create the edges of the multigraph. For example, each transaction could be represented as a tuple (node1, node2, attributes), where node1 and node2 represent the sender and receiver of the transaction, and attributes is a dictionary containing properties such as the amount, timestamp, and transaction type.

2. Create the multigraph: Use the transaction data to create a multigraph using the NetworkX library. The add_edge() method can be used to add edges to the multigraph, where each edge represents a transaction.

3. Extract the edges list and their features: Use the edges() method of the multigraph to extract the edges list and their features, which will be used as input to the GNN.

4. Apply a GNN on the edge list: Use a GNN library such as PyTorch Geometric, Deep Graph Library (DGL) or Spektral to apply a GNN on the edge list. The GNN will learn representations of the edges in the multigraph and use them to classify the edges as fraudulent or non-fraudulent.

5. Evaluation: To evaluate the performance of the GNN, you can split the data into train and test sets, and use the test set to evaluate the accuracy, precision, recall, and F1-score of the model.

### Graph construction

When constructing a graph with transaction edges between card_id and merchant_name, the first step is to identify the nodes in the graph. In this case, the card_id and merchant_name represent the nodes in the graph. Each card_id represents a unique credit card and each merchant_name represents a unique merchant. These nodes can be created by extracting the card_id and merchant_name information from the tabular data and storing them in separate lists.

Once the nodes have been identified, the next step is to create edges between them. These edges represent the transactions that have taken place between a card_id and a merchant_name. To create the edges, a list of transactions is created and for each transaction, an edge is created between the card_id and merchant_name.


### Method 1

We are creating an empty multigraph object called G using the nx.MultiGraph() function from the NetworkX library. Then we add nodes to the graph for each unique card_id and merchant_name from the dataframe df.

The add_nodes_from method is used to add nodes to the graph, it takes an iterable as input and creates a node for each element in the iterable. The df["card_id"].unique() will return a list of unique card_ids in the dataframe, and the df["Merchant Name"].unique will return a list of all the merchant names in the dataframe.

The type attribute is added to each node, it is used to differentiate between card_id and merchant_name nodes. This will help later on when we want to analyze the graph.

**Why did we use a multigraph and not graph?**

The same user (card_id) can buy from the same merchant (Merchant Name) multiple times, so we can have multiple edges between the user and the merchant and for this reason we used multigraph instead of graph

  ![image.png](attachment:7787e1f7-3e13-44c0-9ddb-2e4d67d20c13.png)


In [ ]:
# Create an empty graph
G = nx.MultiGraph()

# Add nodes to the graph for each unique card_id, merchant_name
G.add_nodes_from(df["card_id"].unique(), type='card_id')
G.add_nodes_from(df["Merchant Name"].unique(), type='merchant_name')


The code below adding edges and properties to the edges of a graph. The code iterates through each row of the dataframe, df, and creates a variable for each property then assign it to the edge between the card_id and merchant_name of that row.

In [ ]:
# Add edges and properties to the edges
for _, row in df.iterrows():
    # Create a variable for each properties for each edge

        year = row["Year"],
        month = row["Month"],
        day = row["Day"],
        hour = row["Hour"],
        minute =row["Minute"],
        amount = row["Amount"],
        use_chip =  row["Use Chip"],
        merchant_city = row["Merchant City"],
        errors =  row["Errors?"],
        mcc = row['MCC']


        G.add_edge(row['card_id'], row['Merchant Name'], year = year , month = month , day = day ,
              hour = hour , minute = minute , amount = amount , use_chip = use_chip ,
              merchant_city = merchant_city , errors = errors , mcc = mcc)



In [ ]:
# Get the number of nodes and edges in the graph
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

# Print the number of nodes and edges
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

In [ ]:
# Convert the graph to an adjacency matrix
adj_matrix = nx.adjacency_matrix(G).todense()

In [ ]:
adj_matrix.shape

We will retrieve the properties of a small sample of **nodes** in our graph (G) and print their properties.

In [ ]:
# Get a small sample of the nodes in the graph
sample_nodes = list(G.nodes())[:10]

# Retrieve the properties of the sample nodes
node_properties = nx.get_node_attributes(G, 'type')

# Print the properties of the sample nodes
for node in sample_nodes:
    print(f"Node: {node}, Properties: {node_properties[node]}")


In [ ]:
sample_size = 5
for i, edge in enumerate(G.edges()):
    print(G.get_edge_data(*edge))
    if i >= sample_size - 1:
        break

We will retrieve the properties of a small sample of **edges** in our graph (G) and print their properties.

In [ ]:
# Retrieve the properties errors of all the edges
edge_properties = nx.get_edge_attributes(G, 'errors')

# Count the number of edges by property value
edge_count_by_property = Counter(edge_properties.values())

# Print the count of edges by property value
for property_value, count in edge_count_by_property.items():
    print(f"Property value: {property_value}, Count: {count}")


In [ ]:
# Prepare the data for input into the model
edge_list = list(G.edges(data=True))

In [ ]:
list(edge_list[i][2].values())

We define a PyTorch neural network model called "FraudGNN" which is a type of Graph Neural Network. The model is a simple feedforward neural network with two fully connected (Linear) layers. The first layer has input_dim number of input units and hidden_dim number of output units, while the second layer has hidden_dim number of input units and 1 output unit.
The forward function of the model applies the linear layers to the input tensor "x" and applies a ReLU activation function to the output of the first linear layer.

We also prepare data for input into the model. We define the variable "edge_list" which is a list of edges and their associated data in a graph G. Then we create an empty list called "x" and iterates over each edge in the edge_list. For each edge, it extracts the values of the edge data, converts them to floats if needed, and append them to the list "x". Finally, we convert the list "x" to a PyTorch tensor with float datatype, which is ready to be input into the FraudGNN model.

In [ ]:
class FraudGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FraudGNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x.squeeze(-1)

# Prepare the data for input into the model
edge_list = list(G.edges(data=True))
x = []
for edge in edge_list:
    edge_values = list(edge[2].values())
    edge_values = [float(i[0]) if type(i) == tuple and type(i[0]) == str else i[0] if type(i) == tuple else i for i in edge_values]
    x.append(edge_values)
x = torch.tensor(x, dtype=torch.float)

In [ ]:
target = torch.tensor(df['Is Fraud?'].values, dtype=torch.float)

In [ ]:
# Define the model
input_dim = len(x[0])
hidden_dim = 16
model = FraudGNN(input_dim, hidden_dim)
num_epochs=201

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Train the model
for i in range(num_epochs):
    # Forward pass
    output = model(x)
    # Compute the loss
    loss = criterion(output, target)
    if i % 20 == 0:
        print(f'Epoch: {i}, Loss: {loss.item()}')
    # Zero the gradients
    optimizer.zero_grad()
    # Perform backpropagation
    loss.backward()
    # Update the parameters
    optimizer.step()

### Method 2

In this part we will base our work on the [paper](https://www.sciencedirect.com/science/article/abs/pii/S0957417421017449) **Inductive Graph Representation Learning for fraud detection** written by RafaëlVan Belle, Charles Van Damme, Hendrik Tytgat and JochenDe Weerdt.

Charles has created a python library with the name of inductiveGRL for the experimental setup of their paper.

Library overview :

1. Transaction Data
Any dataset that can be transformed into a graph can be used in our experimental setup. For our research, we used a real-life dataset to construct credit card transaction networks containing millions of transactions. This dataset includes information on the following features: anonymized identification of clients and merchants, merchant category code, country, monetary amount, time, acceptance, and fraud label. This real-life dataset is highly imbalanced and contains only 0.65% fraudulent transactions. Note that the demo data in this repository is artificaly generated for demonstration purposes. The Timeframes component derives the different timeframes for a rolling window setup given a step and window size.

2. Graph Construction
The GraphConstruction component constructs the graphs that will be used by graph representation learners (e.g. FI-GRL and GraphSAGE) to learn node embeddings. We designed the credit card transaction networks as heterogeneous tripartite graphs containing client, merchant and transaction nodes. Because of this tripartite setup, representations can be learned for the transaction nodes. Only the transaction nodes are configured with node features.

3. GraphSAGE
The HinSAGE code deploys a supervised, heterogeneous implementation of the GraphSAGE framework called HinSAGE, to learn embeddings of the transaction nodes in the aforementioned graphs.

4. FI-GRL
The FIGRL code learns embeddings of the transaction nodes in the aforementioned graphs using the Fast Inductive Graph Representation Learning Framework. We call the Matlab implementation of FI-GRL from our Jupyter notebooks, which requires an appropriate installation of matlab.engine in the same folder as the notebooks. If you wish to run FI-GRL from Python, please run the following command in Matlab:

cd (fullfile(matlabroot,'extern','engines','python'))
system('python setup.py install')

This will generate a folder in matlabroot\extern\engines\python\build\lib called 'matlab' please copy this folder and place it on the same location as the notebook from which you want to call matlab.engine. If you don't know your matlab root, running 'matlabroot' in Matlab will return the appropriate path.

5. Classifier
The penultimate component in our pipeline uses the transaction node embeddings to classify the transaction nodes as fraudulent or legitimate. We chose to rely on XGBoost as a classification model, but other classifiers can easily be implemented.

6. Evaluation
The Evaluation component contains functions for the Lift score, Lift curve and precision-recall curve. We focused on these evaluation metrics given the highly imbalanced nature of our dataset. However, this code can easily be extended to contain other evaluation metrics such as ROC plots.

In [ ]:
#library installation
!pip install inductiveGRL

In [ ]:
# necessary imports of this part
from inductiveGRL.graphconstruction import GraphConstruction
from inductiveGRL.hinsage import HinSAGE_Representation_Learner

In [ ]:
# Global parameters:
embedding_size = 64
add_additional_data = True

We will split our data 70% to train_data and 30% for inductive data that will be used to test the model's ability to generalize to new, unseen data.

First we calculate the cutoff point for the split using the formula "cutoff = round(0.7*len(df))" where 0.7 represents 70% of the data, and len(df) is the total number of rows in the dataframe. The round function is used to round the cutoff point to the nearest whole number.

Second we assign the first "cutoff" rows of the dataframe to the variable "train_data" by calling the head() function on the dataframe and passing the cutoff point as an argument. These rows will be used as the training data.

Last we assign the remaining rows of the dataframe (the rows after the cutoff point) to the variable "inductive_data" by calling the tail() function on the dataframe and passing the number of remaining rows (len(df) - cutoff) as an argument. These rows will be used as the inductive data.

In [ ]:
# we will take 70% of our dataset as traindata
cutoff = round(0.7*len(df))
train_data = df.head(cutoff)
inductive_data = df.tail(len(df)-cutoff)

In [ ]:
print('The distribution of fraud for the train data is:\n', train_data['Is Fraud?'].value_counts())
print('The distribution of fraud for the inductive data is:\n', inductive_data['Is Fraud?'].value_counts())

#### Graphe construction

We will create a graph from a train_data using the GraphConstruction class. First we create the dataframes for the node data, second we define the nodes and edges, third  we create a dictionary of features, and finally we create a graph object, and calls the get_stellargraph() method to get the StellarGraph object.

In [ ]:
transaction_node_data = train_data.drop("card_id", axis=1).drop("Merchant Name", axis=1).drop('Is Fraud?', axis=1)
client_node_data = pd.DataFrame([1]*len(train_data["card_id"].unique())).set_index(train_data["card_id"].unique())
merchant_node_data = pd.DataFrame([1]*len(train_data["Merchant Name"].unique())).set_index(train_data["Merchant Name"].unique())

nodes = {"client":train_data.card_id, "merchant":train_data["Merchant Name"], "transaction":train_data.index}
edges = [zip(train_data.card_id, train_data.index),zip(train_data["Merchant Name"], train_data.index)]
features = {"transaction": transaction_node_data, 'client': client_node_data, 'merchant': merchant_node_data}

graph = GraphConstruction(nodes, edges, features)
S = graph.get_stellargraph()
print(S.info())

In [ ]:
#GraphSAGE parameters
num_samples = [2,32]
embedding_node_type = "transaction"

hinsage = HinSAGE_Representation_Learner(embedding_size, num_samples, embedding_node_type)
trained_hinsage_model, train_emb = hinsage.train_hinsage(S, list(train_data.index), train_data['Is Fraud?'], batch_size=5, epochs=10)

## Bibliogtaphy :

**Papers**

Inductive Graph Representation Learning for fraud detection (paper)  : [Link](https://www.sciencedirect.com/science/article/abs/pii/S0957417421017449)

Intelligent Financial Fraud Detection Practices: An
Investigation : [Link](https://arxiv.org/ftp/arxiv/papers/1510/1510.07165.pdf)

Modeling Relational Data with Graph Convolutional Networks : [Link](https://arxiv.org/pdf/1703.06103.pdf)

For more papers check this github [link](https://github.com/benedekrozemberczki/awesome-fraud-detection-papers)

**Library**

Inductive-Graph-Representation-Learning-for-Fraud-Detection (library) : [Link](https://github.com/Charlesvandamme/Inductive-Graph-Representation-Learning-for-Fraud-Detection)

**Dataset**

IBM dataset :
* [Link1](https://github.com/IBM/TabFormer) github
* [Link2](https://ibm.ent.box.com/v/tabformer-data) IBM@Box
* [Link3](https://www.kaggle.com/datasets/ealtman2019/credit-card-transactions) kaggle

**Others**

11 Categorical Encoders and Benchmark : [Link](https://www.kaggle.com/code/subinium/11-categorical-encoders-and-benchmark#1.-Label-Encoder-(LE),-Ordinary-Encoder(OE))

StellarGraph : [Link](https://stellargraph.readthedocs.io/en/stable/README.html#example-gcn)

NetworkX : [Link](https://networkx.org)